In [102]:
##### PTT 股票版 

### 取得首頁 source code
import requests
from bs4 import BeautifulSoup as bs 

url = "https://www.ptt.cc/bbs/Stock/index.html"
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(url,headers=headers)
soup = bs(res.text,"lxml")

In [103]:
### 抓取首頁文章連結
links = []
for a_tag in soup.select("div#main-container div.r-ent div.title a"):
    
    # 過濾 版規 & 盤後閒聊 / 盤中閒聊
    title = a_tag.text
    
    if "股票板板規" in title or "盤後閒聊" in title or "盤中閒聊" in title :
        continue # 跳過此步, 執行下一動迴圈
    else:
        url = "https://www.ptt.cc" + a_tag["href"]
        print(url,title)

https://www.ptt.cc/bbs/Stock/M.1636447699.A.10E.html [標的] 康普、美琪瑪
https://www.ptt.cc/bbs/Stock/M.1636447991.A.83A.html [情報] 力積電10月合併營收 61.65億 月增
https://www.ptt.cc/bbs/Stock/M.1636448546.A.881.html [情報] 3017 奇鋐 110年10月營收40.32億、年增15.49%
https://www.ptt.cc/bbs/Stock/M.1636449650.A.C24.html [新聞] 解封效益顯現 街口投信：股市、油市樂觀
https://www.ptt.cc/bbs/Stock/M.1636450580.A.B7F.html [情報] 2615萬海 Q3
https://www.ptt.cc/bbs/Stock/M.1636451024.A.198.html [情報] 三晃(1721)-11010-營業收入資訊
https://www.ptt.cc/bbs/Stock/M.1636451065.A.179.html [新聞] NVIDIA 推出人工智慧虛擬化身平台
https://www.ptt.cc/bbs/Stock/M.1636451180.A.BF7.html [情報] 3545 敦泰10月營收
https://www.ptt.cc/bbs/Stock/M.1636451227.A.9E7.html [情報] 華南金(2880) 10月自結
https://www.ptt.cc/bbs/Stock/M.1636451480.A.524.html [其他]永豐大戶投神奇開戶歷程（黑特）
https://www.ptt.cc/bbs/Stock/M.1636451503.A.7C2.html [情報] 6509聚和 Q1Q2Q3
https://www.ptt.cc/bbs/Stock/M.1636451968.A.57E.html [新聞] 光洋科經營權之爭 馬堅勇爆料：台積電高度關切
https://www.ptt.cc/bbs/Stock/M.1636453704.A.10B.html [情報] 1109 分點重壓股
https://www.ptt.cc/bb

In [104]:
### 抓取 分頁文章 連結

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

for i in range(1,6):
    
    # 建構 '上頁' 連結
    link = soup.select("div#action-bar-container div.btn-group-paging a")[1]["href"]
    previous_link = "https://www.ptt.cc" + link

    res = requests.get(previous_link,headers=headers)
    soup = bs(res.text,"lxml")

    for a_tag in soup.select("div#main-container div.r-ent div.title a"):

        # 過濾 版規 & 盤後閒聊 / 盤中閒聊
        title = a_tag.text

        if "股票板板規" in title or "盤後閒聊" in title or "盤中閒聊" in title :
            continue # 跳過此步, 執行下一動迴圈
        else:
            url = "https://www.ptt.cc" + a_tag["href"]
            links.append(url)
            
    print("{} is ok.".format(previous_link))
        
print("Done.")

https://www.ptt.cc/bbs/Stock/index5002.html is ok.
https://www.ptt.cc/bbs/Stock/index5001.html is ok.
https://www.ptt.cc/bbs/Stock/index5000.html is ok.
https://www.ptt.cc/bbs/Stock/index4999.html is ok.
https://www.ptt.cc/bbs/Stock/index4998.html is ok.
Done.


In [111]:
# links

In [106]:
### 抓取文章本文 source code 
url = links[11]
print(url)

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res2 = requests.get(url,headers=headers)
soup2 = bs(res2.text,"lxml")

https://www.ptt.cc/bbs/Stock/M.1636446231.A.ACE.html


In [107]:
### 抓取本文的 作者 , 看板 , 標題 , 時間 
span_tags = soup2.select("div#main-content span.article-meta-value")

# 作者
author = span_tags[0].text
print("author :",author)

# 看板
category = span_tags[1].text
print("category :",category)

# 標題
title = span_tags[2].text
print("title :",title)

# 時間
time = span_tags[3].text
print("time :",time)

# span_tags

author : centaurjr (魔術師)
category : Stock
title : [新聞] 威剛第3季獲利銳減97.6% DRAM現貨價下跌
time : Tue Nov  9 16:23:49 2021


In [110]:
### 抓取本文的 內容 , 回應
# 標籤拔除 --> .extract()
# 為了標籤拔除 , 故先抓取 回應資料

# push_tags = soup2.select("div#main-content div.push")

# 抓其中一項做測試
# ele = push_tags[77]
# span_tags = ele.select("span")

# resp = {
#     "tag"     : span_tags[0].text.strip(),
#     "author"  : span_tags[1].text.strip(),
#     "content" : span_tags[2].text.replace(": ","").strip(), 
#     "time"    : span_tags[3].text.strip()
# }
# print(resp)
# span_tags

#############
# for-loop 處理全部回應
# resp_data = []

# for ele in push_tags:
#     span_tags = ele.select("span")

#     resp = {
#         "tag"     : span_tags[0].text.strip(),
#         "author"  : span_tags[1].text.strip(),
#         "content" : span_tags[2].text.replace(": ","").strip(), 
#         "time"    : span_tags[3].text.strip()
#     }
    
#     resp_data.append(resp)
    
#############
# 加入拔除標籤動作
def get_resp_data(ele):
    span_tags = ele.select("span")
    return {
        "tag"     : span_tags[0].text.strip(),
        "author"  : span_tags[1].text.strip(),
        "content" : span_tags[2].text.replace(": ","").strip(), 
        "time"    : span_tags[3].text.strip()
    }


push_tags = soup2.select("div#main-content div.push")
resp_data = []

if len(push_tags) >0:
    
    for ele in push_tags:
        ele.extract()  # 宣告從 div#main-content 中,拔除 div.push 標籤
        
        resp = get_resp_data(ele)
        
        resp_data.append(resp)
        
#         span_tags = ele.select("span")
        
#         resp = {
#             "tag"     : span_tags[0].text.strip(),
#             "author"  : span_tags[1].text.strip(),
#             "content" : span_tags[2].text.replace(": ","").strip(), 
#             "time"    : span_tags[3].text.strip()
#         }

    
# print(resp_data)

In [109]:
### 內容
# print(soup2.select("div#main-content")[0].text)

# 1. 移除標籤
# - div.article-metaline
# - div.article-metaline-right
# - span.f2

def remove_dirty_tag(soup):
    
    # 若存在 , 則移除標籤
    if len(soup.select("div.article-metaline")) >0 :
        
        # 標籤可能多項 , 使用 for-loop 移除
        for tag in soup.select("div.article-metaline"):
            tag.extract()
            
    if len(soup.select("div.article-metaline-right")) >0 :
        for tag in soup.select("div.article-metaline-right"):
            tag.extract()
            
    if len(soup.select("span.f2")) >0 :
        for tag in soup.select("span.f2"):
            tag.extract()
    
    return soup 



soup2 = remove_dirty_tag(soup2)

# 2. 抓取文字
content = soup2.select("div#main-content")[0].text.strip()
print("content:", content)

content: 原文標題： 威剛第3季獲利銳減97.6% DRAM現貨價下跌影響

原文連結： https://reurl.cc/6DelXk

發布時間： 2021年11月9日 週二 下午3:58


原文內容：

（中央社記者張建中新竹2021年11月9日電）動態隨機存取記憶體（DRAM）現貨價下跌，
衝擊記憶體模組廠威剛 (3260) 第3季獲利表現，歸屬母公司淨利新台幣3514萬元，季減
97.6%，每股純益0.14元。

威剛董事會今天通過第3季財報，季營收96.84億元，較第2季減少8.9%，毛利率10.5%，較
第2季下滑7.7個百分點，歸屬母公司淨利3514萬元，季減97.6%，每股純益0.14元。

威剛表示，DRAM現貨價格下跌，是影響第3季毛利率及獲利下滑的主因。

據威剛公布財報數字，前3季營收294.11億元，為歷年同期次高水準，年增25.89%，毛利
率維持16%水準，歸屬母公司淨利21.94億元，年增逾1倍，每股純益9.03元。

展望未來，威剛表示，DRAM上游庫存狀況相當穩定健康，預期第4季DRAM價格跌幅應有限
，毛利率可望回升。


心得/評論：                             ※必需填寫滿20字

營收嚇死人,獲利笑死人

明天應該沒有電梯向哪邊的問題吧?

先猜跳空5%往下


--

Q2 營業利益 1,033
   業外收支   874

腰斬再腰斬我可以理解,但是2.4% ??


In [116]:
### 組合程式

## 引用套件
import requests
from bs4 import BeautifulSoup as bs 

## 定義 function

# 移除標籤
# - div.article-metaline
# - div.article-metaline-right
# - span.f2
def remove_dirty_tag(soup):
    
    # 若存在 , 則移除標籤
    if len(soup.select("div.article-metaline")) >0 :
        
        # 標籤可能多項 , 使用 for-loop 移除
        for tag in soup.select("div.article-metaline"):
            tag.extract()
            
    if len(soup.select("div.article-metaline-right")) >0 :
        for tag in soup.select("div.article-metaline-right"):
            tag.extract()
            
    if len(soup.select("span.f2")) >0 :
        for tag in soup.select("span.f2"):
            tag.extract()
    
    return soup 

# 回應資料
def get_resp_data(ele):
    span_tags = ele.select("span")
    return {
        "tag"     : span_tags[0].text.strip(),
        "author"  : span_tags[1].text.strip(),
        "content" : span_tags[2].text.replace(": ","").strip(), 
        "time"    : span_tags[3].text.strip()
    }


def get_data(soup,url):
    ### 抓取本文的 作者 , 看板 , 標題 , 時間 
    span_tags = soup.select("div#main-content span.article-meta-value")

    # 作者
    author = span_tags[0].text

    # 看板
    category = span_tags[1].text

    # 標題
    title = span_tags[2].text

    # 時間
    time = span_tags[3].text

    ### 抓取本文的 內容 , 回應
    push_tags = soup.select("div#main-content div.push")
    resp_data = []

    if len(push_tags) >0:

        for ele in push_tags:
            ele.extract()  # 宣告從 div#main-content 中,拔除 div.push 標籤

            resp = get_resp_data(ele)

            resp_data.append(resp)

    ### 內容
    soup = remove_dirty_tag(soup)
    content = soup.select("div#main-content")[0].text.strip()
    
    return {
        "author" : author,
        "category" : category,
        "title" : title,
        "time" : time,
        "resp_data" : resp_data,
        "content" : content,
        "url"     : url   # 新增連結欄位
    }



## main 程式

## 取得本文 source code
url = "https://www.ptt.cc/bbs/Stock/index.html"
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(url,headers=headers)
soup = bs(res.text,"lxml")

links = []

print("* 取得本文 source code 完成！")


### 抓取首頁文章連結
for a_tag in soup.select("div#main-container div.r-ent div.title a"):
    
    # 過濾 版規 & 盤後閒聊 / 盤中閒聊
    title = a_tag.text
    
    if "股票板板規" in title or "盤後閒聊" in title or "盤中閒聊" in title :
        continue # 跳過此步, 執行下一動迴圈
    else:
        url = "https://www.ptt.cc" + a_tag["href"]
        links.append(url)
        
print("* 抓取首頁文章連結 完成！")

        
### 抓取 分頁文章 連結
for i in range(1,6):
    
    # 建構 '上頁' 連結
    link = soup.select("div#action-bar-container div.btn-group-paging a")[1]["href"]
    previous_link = "https://www.ptt.cc" + link

    res = requests.get(previous_link,headers=headers)
    soup = bs(res.text,"lxml")

    for a_tag in soup.select("div#main-container div.r-ent div.title a"):

        # 過濾 版規 & 盤後閒聊 / 盤中閒聊
        title = a_tag.text

        if "股票板板規" in title or "盤後閒聊" in title or "盤中閒聊" in title :
            continue # 跳過此步, 執行下一動迴圈
        else:
            url = "https://www.ptt.cc" + a_tag["href"]
            links.append(url)
            
    print("{} is ok.".format(previous_link))

print("* 抓取分頁文章連結 完成！")


### 抓取文章本文 source code + 清理資料

# url = links[11]

dataList = []
for url in links[:10]:      ### 教學用 , 先限定10筆
    res2 = requests.get(url,headers=headers)
    soup2 = bs(res2.text,"lxml")

    # 透過 get_data 從 soup2 解析出 dict 資料
    data = get_data(soup2,url)
    dataList.append(data)

    print("{} is ok.".format(url))
    
print("* 抓取文章資料 完成！")

print("Done.")

* 取得本文 source code 完成！
* 抓取首頁文章連結 完成！
https://www.ptt.cc/bbs/Stock/index5002.html is ok.
https://www.ptt.cc/bbs/Stock/index5001.html is ok.
https://www.ptt.cc/bbs/Stock/index5000.html is ok.
https://www.ptt.cc/bbs/Stock/index4999.html is ok.
https://www.ptt.cc/bbs/Stock/index4998.html is ok.
* 抓取分頁文章連結 完成！
https://www.ptt.cc/bbs/Stock/M.1636447699.A.10E.html is ok.
https://www.ptt.cc/bbs/Stock/M.1636447991.A.83A.html is ok.
https://www.ptt.cc/bbs/Stock/M.1636448546.A.881.html is ok.
https://www.ptt.cc/bbs/Stock/M.1636449650.A.C24.html is ok.
https://www.ptt.cc/bbs/Stock/M.1636450580.A.B7F.html is ok.
https://www.ptt.cc/bbs/Stock/M.1636451024.A.198.html is ok.
https://www.ptt.cc/bbs/Stock/M.1636451065.A.179.html is ok.
https://www.ptt.cc/bbs/Stock/M.1636451180.A.BF7.html is ok.
https://www.ptt.cc/bbs/Stock/M.1636451227.A.9E7.html is ok.
https://www.ptt.cc/bbs/Stock/M.1636451480.A.524.html is ok.
* 抓取文章資料 完成！
Done.


In [118]:
# print(len(dataList))
# dataList[5]